In [ ]:
# Get the path to the desktop directory on macOS
import os
user_directory = os.path.join(os.path.expanduser('~'), '')
os.chdir(user_directory)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import librosa.display
import numpy as np
from scipy.linalg import solve_toeplitz, toeplitz
import scipy

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import os

def compute_mfcc_and_deltas(y, sr, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    
    # If data is too small, return zeros for delta and delta2
    if mfccs.shape[1] < 3:
        return mfccs, np.zeros_like(mfccs), np.zeros_like(mfccs)
    
    delta_width = 9
    while delta_width > mfccs.shape[1]:
        delta_width -= 2
    
    # Ensure delta_width is at least 3 and remains odd
    delta_width = max(3, delta_width)

    delta_mfccs = librosa.feature.delta(mfccs, width=delta_width)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2, width=delta_width)
    return mfccs, delta_mfccs, delta2_mfccs


def main(audio_path, excel_path, save_directory):
    y, sr = librosa.load(audio_path, sr=None)
    df = pd.read_csv(excel_path)
    counter = 0
    for index, row in df.iterrows():
        counter = counter + 1
        start_sample = librosa.time_to_samples(row['start'], sr=sr)
        end_sample = librosa.time_to_samples(row['end'], sr=sr)
        word_segment = y[start_sample:end_sample]

        mfccs, delta_mfccs, delta2_mfccs = compute_mfcc_and_deltas(word_segment, sr)

        # Skip plotting and saving for words with fewer than 3 MFCC frames
        if mfccs.shape[1] < 3:
            continue
        
        word_label = row.get('word', f'Word_{index+1}')  # Assumes there's a 'word' column. If not, uses default label.

        plt.figure(figsize=(10, 6))
        plt.subplot(3, 1, 1)
        librosa.display.specshow(mfccs, x_axis='time')
        plt.title(f'MFCCs of Word: {word_label}')
        plt.colorbar()
        
        plt.subplot(3, 1, 2)
        librosa.display.specshow(delta_mfccs, x_axis='time')
        plt.title(f'Delta MFCCs of Word: {word_label}')
        plt.colorbar()
        
        plt.subplot(3, 1, 3)
        librosa.display.specshow(delta2_mfccs, x_axis='time')
        plt.title(f'Delta-Delta MFCCs of Word: {word_label}')
        plt.colorbar()
        
        plt.tight_layout()
        plt.savefig(f"{save_directory}{str(counter)}_{word_label}.jpg", dpi=300)
        plt.close()


if __name__ == "__main__":
    audio_path = './Desktop/Folders/.wav'  # Replace with your audio file path
    excel_path = './Desktop/Folders/.csv'  # Replace with your .csv file path
    subject = 'Erfan'
    save_directory = './Desktop/Folders/Frequency_plots/MFCC/' + subject + '/'  # Specify the directory where you want to save the plots
    
    main(audio_path, excel_path, save_directory)
